## **Limpieza de los archivos Excel de "Emergencias UNGRD"**
----

#### ***Importación de paquetes***

In [2]:
import pandas as pd

### ***Cargar los datasets mediante Pandas***

In [3]:
df_2003 = pd.read_excel('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2003.xls', sheet_name='REPORTE DE EMERGENCIAS', skiprows=1)

## **Procesamiento de los datos**
---


### ***Declarar función para realizar la limpieza***

In [4]:
def limpiar_datos_emergencias(filepath, end_row, sheet_name='REPORTE DE EMERGENCIAS', skip_rows=1):
    # Leer el archivo Excel
    df = pd.read_excel(filepath, sheet_name=sheet_name, skiprows=skip_rows)
    
    # Seleccionar filas y columnas relevantes
    df = df[0:end_row]  # Restar 1 y el valor de skip_rows para que incluya la fila deseada
    df = df[["FECHA", "DEPTO", "MUNICIPIO", "EVENTO", "MUERTOS", "HERIDOS", "DESAPA.", "PERSONAS", "FAMILIAS", "VIV.DESTRU.", "VIAS", "HECTAREAS"]]
    
    # Rellenar valores faltantes con cero y convertir tipos de datos
    df.fillna(0, inplace=True)
    df = df.astype({
        'MUERTOS': int, 
        'HERIDOS': int, 
        'DESAPA.': int, 
        'PERSONAS': int, 
        'FAMILIAS': int, 
        'VIV.DESTRU.': int, 
        'VIAS': int, 
        'HECTAREAS': int
    })
    
    # Convertir la columna de fechas y extraer el año y el mes
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    df['AÑO'] = df['FECHA'].dt.year
    df['MES'] = df['FECHA'].dt.month
    
    # Reorganizar las columnas
    cols = ['AÑO', 'MES'] + [col for col in df.columns if col not in ['AÑO', 'MES']]
    df = df[cols]
    
    # Renombrar columnas
    df = df.rename(columns={
        'DEPTO': 'DEPARTAMENTO',
        'MUERTOS': 'FALLECIDOS',
        'DESAPA.': 'DESAPARECIDOS',
        'VIV.DESTRU.': 'VIVIENDAS'
    })
    
    return df

### ***Procesar CSVs***
---

#### ***2003***

In [5]:
df_2003 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2003.xls', 561)

In [6]:
# df_2003.head()
# df_2003.tail()
df_2003["EVENTO"].unique()

array(['INUNDACION', 'VENDAVAL', 'INCENDIO FORESTAL',
       'INCENDIO ESTRUCTURAL', 'CONTAMINACION', 'SISMO', 'DESLIZAMIENTO',
       'AVALANCHA', 'ACCIDENTE AEREO', 'ACCIDENTE', 'SEQUIA', 'VARIOS',
       'GRANIZADA', 'ANTROPICO', 'COLAPSO ESTRUCTURAL', 'ERUPCION ',
       'EXPLOSION'], dtype=object)